# Scraping Scovilles


Here I will be scraping the scoville data found here: http://www.scottrobertsweb.com/scoville-scale/

I need to find the HTML tags in the table that correspond to the columns 'Sauce/Pepper' and 'Scoville Units'

In [11]:
import urllib.request
from bs4 import BeautifulSoup



web_page = "http://www.scottrobertsweb.com/scoville-scale/"

page = urllib.request.urlopen(web_page)
soup = BeautifulSoup(page)

#print(soup.prettify())

/usr/local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [12]:
## easy enough, beautiful soup has a way to directly pull out the html for tables
table = soup.table
## printing the table is too much space for github, but it's how I found the elements to select
## print(table)

In [13]:
## this searches through the table for the 'tr' tags with a 'bgcolor' of #efefef
for row in table.find_all('tr', bgcolor="#efefef"):
    td = row.find_all('td', class_ = "sauce")
    #print(td)

In [14]:
df_rows = []
## building off of the previous block, I want to pull out the three columns of interest
## and put them in a list called 'df_rows'
for row in table.find_all('tr', bgcolor="#efefef"):
    cols       = row.find_all('b') # column one, two, three
    text_cols  = [x.find(text = True) for x in cols]
    df_rows.append(text_cols)

print(df_rows[0:5]) # look at the list


[['Blair’s 16 Million Reserve', '16,000,000', '3200 times'], ['Blair’s 6 A.M.', '16,000,000', '3200 times'], ['Blair’s 2009 Halloween Reserve', '16,000,000', '3200 times'], ['Blair’s 2005 Holiday Reserve', '14,725,000', '2945 times'], ['Meaty Vegan’s Experiment 1064', '14,200,000', '2840 times']]


I need to make scoville and jalapeno_comp numeric. 

This could be weird, since some of the scovilles have ranges. I'll just make them an array in the data table. I also need to remove question marks.

For jalapeno comp I need to remove the 'times' and change the percents to decimals.

I'm not sure how to do either in python, let alone pandas but I will try.


In [15]:
import re

df_rows_fix = []

for x in df_rows: 
    ## fix the third entry in the list to remove the times and % characters, and make it a float
    if len(x) == 3:
        if x[2].endswith(' times'):
            x2 = float(re.sub(' times$', '', x[2]))
        if x[2].endswith("%"):
            x_temp = re.sub('%$', '', x[2])
            x2     = float("0." + x_temp) # convert the percentage to a factor of jalapeno heat
            
    else:
        x2 = 0 # jalapeno reference point
    
    ## fix the second entry in the list to remove the ranges
    
    x1_comma    = re.sub(',', '', x[1])    
    multi_words = re.findall('[0-9]+', x1_comma)

    if len(multi_words) > 1:
        scoville_start = int(multi_words[0])
        scoville_end   = int(multi_words[len(multi_words)-1])
        x1 = str(scoville_start) + "_" + str(scoville_end)
    else:
        x1 = multi_words[0] + "_" + multi_words[0]
    
    ## append the list to the new list of rows
    df_rows_fix.append([x[0], x1, x2])
    

In [16]:

import pandas as pd
## turn it all into a data frame
df = pd.DataFrame([x for x in df_rows_fix], columns=['name', 'scoville', 'jalapeno_comp'])

df.head(10)



,name,scoville,jalapeno_comp
0,Blair’s 16 Million Reserve,16000000_16000000,3200.0
1,Blair’s 6 A.M.,16000000_16000000,3200.0
2,Blair’s 2009 Halloween Reserve,16000000_16000000,3200.0
3,Blair’s 2005 Holiday Reserve,14725000_14725000,2945.0
4,Meaty Vegan’s Experiment 1064,14200000_14200000,2840.0
5,Blair’s 2004 Holiday Reserve,14100000_14100000,2820.0
6,Blair’s 2006 Holiday Reserve,14000000_14000000,2800.0
7,Blair’s 2005 Halloween Reserve,13500000_13500000,2700.0
8,Blair’s 2007 Halloween Reserve,13500000_13500000,2700.0
9,Blair’s 2006 Halloween Reserve,13500000_13500000,2700.0


In [17]:
df.describe()

,jalapeno_comp
count,220.000000
mean,267.416864
std,658.374576
min,0.000000
25%,0.900000
50%,20.000000
75%,183.500000
max,3200.000000


In [18]:

## export the table to a csv
df.to_csv(path_or_buf='scoville.csv', index=False)